In [1]:
import xml.etree.ElementTree as ET
import os
import numpy as np
import re
import copy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

# Parsing input files

Parsed input files from xml downloaded locally

In [2]:
# A case is an xml file (related to the Australian case)
class Case:
    def __init__(self, xmlFolder):
        self.xmlFolder = xmlFolder
        self.name = ""
        self.sentences = []
        self.catchphrases = []
        self.AustLII = ""
        
    def parseXML(self):
        tree = ET.parse(self.xmlFolder)
        self.root = tree.getroot()
        self.name = self.root.findall("name")[0].text
        self.AustLII = self.root.findall("AustLII")[0].text
        self.getCatchPhrases()
        self.getSentences()
        
    def getCatchPhrases(self):
        catchphrases_root =  self.root.findall('catchphrases')[0]
        self.catchphrases = []
        for catchphrase in catchphrases_root.findall('catchphrase'):
            self.catchphrases.append(catchphrase.text)
        return self.catchphrases
    def getSentences(self):
        sentences_root =  self.root.findall('sentences')[0]
        self.sentences = []
        for sentence in sentences_root.findall('sentence'):
            self.sentences.append(sentence.text)
        
        self.sentences = self.sentences[:]
    def printSummary(self):
        print(str(case.name))
        print(str(case.AustLII)+"\n")
        print(str(len(case.sentences)) + " Sentences")
        print(str(len(case.catchphrases)) + " Catchphrases")

# Preprocess input Data

In [3]:
class Preprocess:
    @staticmethod
    def normalize(text):
        output = copy.copy(text)
        for idx, t in enumerate(text):  
            t=re.sub("(\\t)", ' ', str(t)).lower()                      #remove escape charecters
            t=re.sub("(\\r)", ' ', str(t)).lower() 
            t=re.sub("(\\n)", ' ', str(t)).lower()
            t=re.sub("(__+)", ' ', str(t)).lower()                      #remove consecutive underscores
            t=re.sub("(~~+)", ' ', str(t)).lower()                      #remove consecutively tildas
            t=re.sub("(\+)", ' ', str(t)).lower()                       #remove all +
            t=re.sub("(\.)", ' ', str(t)).lower()                       #remove all .
            t=re.sub("(\•)", ' ', str(t)).lower()                       #remove all •
            t=re.sub("(\-)", ' ', str(t)).lower()                       #remove all -
            t=re.sub(r"[<>()|&©ø•\[\]\'\",:;?~*!]", ' ', str(t)).lower() #remove <>()|&©ø"',;?~*!
            t=re.sub("(\s+.\s+)", ' ', str(t)).lower()                  #remove words formed of 1 character
            t = re.sub("(\s+)",' ',str(t)).lower()                      #remove multiple spaces
            t=re.sub("(\s+.\s+)", ' ', str(t)).lower()                  #remove words formed of 1 character again
            output[idx] = t
        output = [x.strip(' ') for x in output]
        output = [x for x in output if len(x)>5]
        return output

# Extractive Summarizer

In [4]:
class ExtractiveSummary:
    
    def __init__(self,sentences,catchphrases,nbrOfClusters):
        # Assume sentences and catchphrases are normalized
        self.sentences = sentences
        self.catchphrases = catchphrases
        # Should be equal to number of catchphrases (= to nbr of phrases to output)
        self.nbClust = nbrOfClusters
    
    def GenerateTFIDF(self):
        tf_idf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tf_idf_vectorizer.fit_transform(self.sentences)
        return tfidf_matrix
    
    def KMeans_Cluster(self,tfidf_matrix):
        kmeans_cluster = KMeans(n_clusters=self.nbClust)
        kmeans_cluster.fit(tfidf_matrix)
        clusters_labels = kmeans_cluster.labels_.tolist()
        # return labels of all clusters
        return clusters_labels
    
    def GroupClusterSentences(self,clusters_labels):
        # Dictionary holding each sentence with its corresponding text and cluster assigned
        sentenceInfo = {}
        for i, sentence in enumerate(self.sentences):
            sentenceInfo[i] = {}
            sentenceInfo[i]['text'] = sentence
            sentenceInfo[i]['cluster'] = clusters_labels[i]
        #Dictionary holding each cluster with its corresponding sentences 
        clusters = {}
        for key, sentence in sentenceInfo.items():
            if sentence['cluster'] not in clusters:
                clusters[sentence['cluster']] = []
            clusters[sentence['cluster']].append(sentence['text'])
            sentence['i'] = len(clusters[sentence['cluster']]) - 1
        return sentenceInfo, clusters
    
    def GetCosineSimilarity(self,clusters):
        #For each cluster, get the sentence with the highest 
        # cosine similarity value with the rest of the sentences.
        maxCosineVals = {}
        for key, clusterSentences in clusters.items():
            maxCosineVals[key] = {}
            maxCosineVals[key]['val'] = 0
            vectorizer=TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(clusterSentences)
            cos_sim_matrix = cosine_similarity(tfidf_matrix)
            for i, row in enumerate(cos_sim_matrix):
                sumPerRow = 0
                for val in row:
                    sumPerRow += val
                if sumPerRow > maxCosineVals[key]['val']:
                    maxCosineVals[key]['val'] = sumPerRow
                    maxCosineVals[key]['i'] = i
        return maxCosineVals

    def GetHighestRankingSentences(self,maxCosineVals,sentenceInfo,clusters):
        # For each cluster's get the sentence with the maximum cosine value
        highestRankingSentences_text = []
        highestRankingSentences_Indeces = []
        for key, sentence in maxCosineVals.items():
            clusterIndex = key
            sentenceIndex = sentence['i']
            chosen_Sentence = clusters[clusterIndex][sentenceIndex]
            
            #Search for sentence in the corresponding cluster
            
            for i, sentence in sentenceInfo.items():
                if sentence['cluster'] == clusterIndex and sentence['i'] == sentenceIndex:
                    highestRankingSentences_Indeces.append(i)
                    

        highestRankingSentences_Indeces.sort()
        for i in highestRankingSentences_Indeces:
            highestRankingSentences_text.append(self.sentences[i])
        return highestRankingSentences_text
    
    def GenerateExtractiveSummary(self):
        tfidf_matrix = self.GenerateTFIDF()
        clusters_labels =  self.KMeans_Cluster(tfidf_matrix)
        sentenceInfo, clustersInfo = self.GroupClusterSentences(clusters_labels)
        maxCosineVals = self.GetCosineSimilarity(clustersInfo)
        summary_list = self.GetHighestRankingSentences(maxCosineVals,sentenceInfo,clustersInfo)
  
        return summary_list

# Generate extractive summaries

In [5]:
import glob
from random import sample 

#PARSING DATASET
extractive_summaries=[]
catchphrases=[]
sentences = []

files = glob.glob("fulltext/*.xml")
files = sample(files,len(files))

done = 0

for f in files:
    try:
        if done == 1:
            print("Done")
            break
        full_path = f
        case = Case(full_path)
        case.parseXML()
        #case.printSummary()
        sentences_normalized = Preprocess.normalize(case.sentences);#input sentences
        catchphrases_normalized = Preprocess.normalize(case.catchphrases); #output sentences
        #limit each catchphrase with _START_ and _END_ tags for the model
        #catchphrases_normalized=[ ('xxxstartxxx '+ c + ' xxxendxxx') for c in catchphrases_normalized]

        #generative summarizer
        nbOfClusters = len(catchphrases_normalized)
        extractive_summary = ExtractiveSummary(sentences_normalized,catchphrases_normalized,nbOfClusters)
        extractive_sentences= extractive_summary.GenerateExtractiveSummary()
        
        extractive_summaries+=extractive_sentences
        catchphrases+=catchphrases_normalized
        
        done = done + 1
    except:
        continue


# ROUGE Evaluation for the extractive summarizer vs catchphrases 

In [7]:
from rouge_score import rouge_scorer

r1_precisions = []
r1_recalls = []
r1_f_scores = []

r2_precisions = []
r2_recalls = []
r2_f_scores = []

rl_precisions = []
rl_recalls = []
rl_f_scores = []

for i, ext_sum in enumerate(extractive_summaries):
    system_generated_summary = ext_sum
    manual_summmary = catchphrases[i]


    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=False)
    scores = scorer.score(system_generated_summary,manual_summmary)
    r1_precisions.append(scores['rouge1'].precision)
    r2_precisions.append(scores['rouge2'].precision)
    rl_precisions.append(scores['rougeL'].precision)
    
    r1_recalls.append(scores['rouge1'].recall)
    r2_recalls.append(scores['rouge2'].recall)
    rl_recalls.append(scores['rougeL'].recall)
    
    r1_f_scores.append(scores['rouge1'].fmeasure)
    r2_f_scores.append(scores['rouge2'].fmeasure)
    rl_f_scores.append(scores['rougeL'].fmeasure)
    
    
r1_avg_prec = np.mean(r1_precisions)
r2_avg_prec = np.mean(r2_precisions)
rl_avg_prec = np.mean(rl_precisions)

r1_avg_recall = np.mean(r1_recalls)
r2_avg_recall = np.mean(r2_recalls)
rl_avg_recall = np.mean(rl_recalls)

r1_avg_f_score = np.mean(r1_f_scores)
r2_avg_f_score = np.mean(r2_f_scores)
rl_avg_f_score = np.mean(rl_f_scores)

print("ROUGE 1\n")
print("Precision is :"+str(r1_avg_prec)+"\nRecall is :"+str(r1_avg_recall)+"\nF Score is :"+str(r1_avg_f_score))
print("\n\n")

print("ROUGE 2\n")
print("Precision is :"+str(r2_avg_prec)+"\nRecall is :"+str(r2_avg_recall)+"\nF Score is :"+str(r2_avg_f_score))
print("\n\n")

print("ROUGE L\n")
print("Precision is :"+str(rl_avg_prec)+"\nRecall is :"+str(rl_avg_recall)+"\nF Score is :"+str(rl_avg_f_score))
print("\n\n")

ROUGE 1

Precision is :0.20851570401246075
Recall is :0.052009009748833004
F Score is :0.07231685545057086



ROUGE 2

Precision is :0.041175161485222274
Recall is :0.009233605416359994
F Score is :0.012980729409287543



ROUGE L

Precision is :0.1842840612935458
Recall is :0.04376162629867269
F Score is :0.0613259695214317





# Abstractive Summarizer
BART Model pretrained on the CNN and daily mail data set retreived from huggingface.co

In [9]:
from transformers import pipeline

In [10]:
# Bart model that was fine-tuned on the CNN / Daily Mail data set.
summarizer = pipeline("summarization")

### Input of Abstractive Summarizer is output of Extractive Summarizer

In [11]:
abs_summaries = []
for i, ext in enumerate(extractive_summaries):
    new_summ = summarizer(ext, max_length=130, min_length=30, do_sample=False)
    abs_summaries.append(new_summ[0]["summary_text"])

Your max_length is set to 130, but you input_length is only 85. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 36. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 53. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 36. You might consider

Your max_length is set to 130, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 79. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 97. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 30, but you input_length is only 11. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 130, but you input_length is only 11. You might consider 

Your max_length is set to 130, but you input_length is only 72. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 66. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 74. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 40. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 35. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 16. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 19. You might consider

Your max_length is set to 130, but you input_length is only 88. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 38. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 30, but you input_length is only 7. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 130, but you input_length is only 7. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 38. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 30, but you input_length is only 14. You might consider dec

Your max_length is set to 130, but you input_length is only 16. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 16. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 30, but you input_length is only 14. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 130, but you input_length is only 14. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 26. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 64. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 33. You might consider 

...

Your max_length is set to 130, but you input_length is only 65. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 102. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 114. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 43. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 26. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 67. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 91. You might consid

Your max_length is set to 130, but you input_length is only 13. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 40. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 62. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 55. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 47. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 38. You might consider

Your max_length is set to 130, but you input_length is only 51. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 20. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 113. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 52. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 30, but you input_length is only 9. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 130, but you input_length is only 9. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 69. You might consider d

Your max_length is set to 130, but you input_length is only 47. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 50. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 40. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 21. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 66. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 130, but you input_length is only 34. You might consider

KeyboardInterrupt: 

# ROUGE Evaluation for the abstractive summarizer vs catchphrases 

In [23]:
from rouge_score import rouge_scorer

r1_precisions = []
r1_recalls = []
r1_f_scores = []

r2_precisions = []
r2_recalls = []
r2_f_scores = []

rl_precisions = []
rl_recalls = []
rl_f_scores = []

for i, abs_sum in enumerate(abs_summaries):
    system_generated_summary = abs_sum
    manual_summmary = catchphrases[i]


    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=False)
    scores = scorer.score(system_generated_summary,manual_summmary)
    r1_precisions.append(scores['rouge1'].precision)
    r2_precisions.append(scores['rouge2'].precision)
    rl_precisions.append(scores['rougeL'].precision)
    
    r1_recalls.append(scores['rouge1'].recall)
    r2_recalls.append(scores['rouge2'].recall)
    rl_recalls.append(scores['rougeL'].recall)
    
    r1_f_scores.append(scores['rouge1'].fmeasure)
    r2_f_scores.append(scores['rouge2'].fmeasure)
    rl_f_scores.append(scores['rougeL'].fmeasure)
    
    
r1_avg_prec = np.mean(r1_precisions)
r2_avg_prec = np.mean(r2_precisions)
rl_avg_prec = np.mean(rl_precisions)

r1_avg_recall = np.mean(r1_recalls)
r2_avg_recall = np.mean(r2_recalls)
rl_avg_recall = np.mean(rl_recalls)

r1_avg_f_score = np.mean(r1_f_scores)
r2_avg_f_score = np.mean(r2_f_scores)
rl_avg_f_score = np.mean(rl_f_scores)

print("ROUGE 1\n")
print("Precision is :"+str(r1_avg_prec)+"\nRecall is :"+str(r1_avg_recall)+"\nF Score is :"+str(r1_avg_f_score))
print("\n\n")

print("ROUGE 2\n")
print("Precision is :"+str(r2_avg_prec)+"\nRecall is :"+str(r2_avg_recall)+"\nF Score is :"+str(r2_avg_f_score))
print("\n\n")

print("ROUGE L\n")
print("Precision is :"+str(rl_avg_prec)+"\nRecall is :"+str(rl_avg_recall)+"\nF Score is :"+str(rl_avg_f_score))
print("\n\n")


ROUGE 1

Precision is :0.1882224470875977
Recall is :0.04973307602801065
F Score is :0.07127664517375926



ROUGE 2

Precision is :0.033352135469271856
Recall is :0.00823565134173693
F Score is :0.011786266729553536



ROUGE L

Precision is :0.1671987675837404
Recall is :0.041741060863540014
F Score is :0.06068262818367998





In [40]:
sentences_normalized[0]

'this is an appeal from decision of hartnett fm dismissing an application for judicial review of decision of the refugee review tribunal the tribunal'

In [68]:
ext1 = str(extractive_summaries[242])

In [69]:
abs1 = str(abs_summaries[242])

In [70]:
cat1 = str(catchphrases[242])

In [71]:
print("Extractive Summary\n " + ext1)
print("\n")
print("Abstractive Summary\n " + abs1)
print("\n")
print("Catchphrase\n " + cat1)

Extractive Summary
 in re william vazquez associates federal commissioner of taxation 2005 aata 301 2005 58 atr 1357 where director of the taxpayer said that the taxpayer did not have the funds to make payment of outstanding amounts and that he had been of the mistaken belief as to the time payments were due to be made the commissioner imposed penalty for late lodgement


Abstractive Summary
  in re william vazquez associates federal commissioner of taxation 2005 aata 301 2005 58 atr 1357 . Director of the taxpayer said that the taxpayer did not have the funds to make payment of outstanding amounts .


Catchphrase
 whether it is necessary that there be special circumstances before the remission discretion is exercised
